In [ ]:
import os
print(os.getcwd())
def update_working_directory():
    from pathlib import Path
    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)
update_working_directory()

In [ ]:
import numpy as np
import pandas as pd

from src.data.collect_data import *

In [ ]:
vocab = initiate_vocab()

historical_data, tries_session = initiate_session()

In [ ]:
#while (sum(vocab['succeed_session']) + sum(vocab['succeed'])) != len(vocab):
for i in range(100):
    
    nb_known_words_session = sum(vocab['succeed_session'])

    # Pick a random word
    row_to_test = choose_a_word(vocab)
    if row_to_test == -1:
        break
    
    # Choose language
    input_language, output_language = choose_a_language()

    # Take a guess
    your_guess = prompt_question(vocab, row_to_test, input_language, output_language, nb_known_words_session, tries_session)

    # Answer analysis
    is_it_correct = your_guess == vocab.loc[row_to_test, output_language]
    is_it_another_word = your_guess in vocab[output_language].values

    # Do we want to stop?
    if (your_guess == "STOP IT"):
        print("Your last word was '{}={}'".format(
            vocab.loc[row_to_test, input_language], vocab.loc[row_to_test, output_language]))
        break

    # If correct answer
    if is_it_correct:
        if_correct()

    # If wrong answer
    if not is_it_correct:
        if_not_correct(vocab, row_to_test, input_language, output_language, your_guess, is_it_another_word)

    # Add to historical data
    historical_data = add_historical_data(historical_data, vocab, row_to_test, output_language, is_it_correct, your_guess)

    # Update vocab
    vocab = update_vocab(vocab, row_to_test, is_it_correct)

    tries_session += 1

In [ ]:
# Update and save historical data
update_historical_data(historical_data)

In [ ]:
# Update and save vocab
finalize_vocab(vocab)